# PostgreSQL to S3 Export using Glue/Spark

In [4]:
import sys
import boto3
import json
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

In [5]:
# Initialize Glue context
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=GlueReplApp, master=jes) created by __init__ at /tmp/4543972804912652780:514 


In [6]:
# Get credentials from Secrets Manager
secrets_client = boto3.client('secretsmanager')
secret = secrets_client.get_secret_value(SecretId='rds-read-only')
credentials = json.loads(secret['SecretString'])

In [7]:
# JDBC connection properties using Secrets Manager
jdbc_url = f"jdbc:postgresql://{credentials['host']}:{credentials['port']}/{credentials['dbname']}"
connection_properties = {
    "user": credentials['username'],
    "password": credentials['password'],
    "driver": "org.postgresql.Driver"
}

In [8]:
def print_tables_with_index():
    query = """
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'public' AND table_type = 'BASE TABLE'
    ORDER BY table_name
    """
    
    tables_df = spark.read \
        .format("jdbc") \
        .option("url", jdbc_url) \
        .option("query", query) \
        .option("user", credentials['username']) \
        .option("password", credentials['password']) \
        .option("driver", "org.postgresql.Driver") \
        .load()
    
    all_tables = [row.table_name for row in tables_df.collect()]
    valid_tables = []
    
    for table in all_tables:
        try:
            spark.read \
                .format("jdbc") \
                .option("url", jdbc_url) \
                .option("query", f"SELECT 1 FROM {table} LIMIT 1") \
                .option("user", credentials['username']) \
                .option("password", credentials['password']) \
                .option("driver", "org.postgresql.Driver") \
                .load().collect()
            valid_tables.append(table)
        except:
            pass
    
    print("# Valid tables:")
    for i, table in enumerate(valid_tables):
        padding = max(0, 40 - len(table))
        print(f'    "{table}",{" " * padding}# {i}')
    
    return valid_tables

tables = print_tables_with_index()



# Valid tables:
    "censo_escolar_2024",                      # 0
    "ed_enem_2024_participantes",              # 1
    "ed_enem_2024_resultados",                 # 2
    "ed_enem_2024_resultados_amos_per",        # 3
    "ed_superior_cursos",                      # 4
    "ed_superior_ies",                         # 5
    "educacao_basica",                         # 6
    "municipio",                               # 7
    "municipio_ride_brasilia",                 # 8
    "ocorrencia",                              # 9
    "pib_municipios",                          # 10
    "regiao",                                  # 11
    "sus_aih",                                 # 12
    "sus_procedimento_ambulatorial",           # 13
    "unidade_federacao",                       # 14


In [9]:
table_name = tables[0]
print("tbl:", table_name)

tbl: censo_escolar_2024


In [28]:
# Usage with selected table
df = spark.read.jdbc(
        url=jdbc_url,
        table=table_name,
        properties=connection_properties
    )

df.sample(.1).toPandas().head()

   NU_ANO_CENSO NO_REGIAO  ...  QT_TUR_FUND_AF_INT QT_TUR_MED_INT
0          2024   Sudeste  ...                 NaN            NaN
1          2024   Sudeste  ...                 NaN            NaN
2          2024   Sudeste  ...                 0.0            0.0
3          2024   Sudeste  ...                 0.0            0.0
4          2024       Sul  ...                 0.0            0.0

[5 rows x 426 columns]


In [29]:
df.write.mode("overwrite").parquet(f"s3://iesb-bigdata/bronze/{table_name}/")
        
print(f"Table {table_name}")

Table censo_escolar_2024


In [ ]:
for table_name in tables:
    df = spark.read.jdbc(
        url=jdbc_url,
        table=table_name,
        properties=connection_properties
    )

    df.write.mode("overwrite").parquet(f"s3://iesb-bigdata/bronze/{table_name}/")
    print(f"Table {table_name}")